In [1]:
import arcpy
import os

In [8]:
#For code reference f_storm_surge.ipynb
global_dem = r"G:\CASA\ejen\Future Storm Surge\dem_files\global cities lecz dem\global_cities_lecz_dem.tif"

In [9]:
#Interpolate RP10 storm surge gauge values across LECZ extent
interpolated_storm_surge = r"G:\CASA\output_rasters\COAST StormTide Return Periods\COAST_st_rp_0010.tif"
coast_variables_shape = r"G:\CASA\data\COAST-RP\points\COAST_RP_allvar.shp"
outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]' 
snapRaster=global_dem
extent=global_dem
cellSize=global_dem
mask=global_dem
arcpy.sa.IDW(
    in_features=coast_variables_shape,
    z_field="st_rp_0100",
    out_raster= interpolated_storm_surge,
    cell_size= global_dem,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=99.342614921292 S_MINOR=99.342614921292 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

In [4]:
#Difference elevation and surge values to identify flooded versus not flooded areas
from arcpy.ia import *
outMinus = Minus(interpolated_storm_surge, global_dem)
difference_raster = r"G:\CASA\output_rasters"
arcpy.env.workspace = difference_raster
outMinus.save("difference_storm_surge_rp10.tif")

In [2]:
difference_storm_surge = r"G:\CASA\output_rasters\difference_storm_surge_rp10.tif"

In [8]:
from arcpy.sa import *
#Create binary mask for areas where elevation is lower than interpolated storm surge values
binary_raster = Con(Raster(difference_storm_surge) > 0, 1)
mask_raster = r"G:\CASA\output_rasters"
arcpy.env.workspace = mask_raster
binary_raster.save("recent_storm_surge_rp10_binary.tif")

In [12]:
cities_dataset = r"G:\CASA\ejen\cities_dataset_progress_7_18.gpkg"
arcpy.env.workspace = cities_dataset
datasets = arcpy.ListFeatureClasses()
cities_boundary = f"{cities_dataset}\{datasets[0]}"

In [15]:
#Keep binary mask values only where cities are present
from arcpy.ia import *
arcpy.env.mask = cities_boundary
in_raster = Raster(r"G:\CASA\output_rasters\recent_storm_surge_rp10_binary.tif")
outCon = Con(in_raster == 1, 1)  # Set values where condition is true to 1
outRaster = Con(IsNull(outCon), 0, outCon)
outRaster.save("G:/CASA/ejen/Recent Storm Surge/outraster.tif")

In [ ]:
#Expand rasters to improve alignment with other indicators after turning into vectors
out_raster = arcpy.sa.Expand(
    in_raster="outraster.tif",
    number_cells=2,
    zone_values=[1,0],
    expand_method="MORPHOLOGICAL"
)
out_raster.save(r"G:\CASA\ejen\casa_project\Default.gdb\Expand_outra1")

In [ ]:
#Convert raster to polygon
arcpy.conversion.RasterToPolygon(
    in_raster="Expand_outra1",
    out_polygon_features=r"G:\CASA\ejen\casa_project\Default.gdb\RasterT_Expand_6",
    simplify="NO_SIMPLIFY",
    raster_field="Value",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

In [ ]:
#Interset surge and cities polygons
arcpy.analysis.PairwiseIntersect(
    in_features="RasterT_Expand_6;cities_dataset_final",
    out_feature_class=r"G:\CASA\ejen\casa_project\Default.gdb\RasterT_Expand_PairwiseInter5",
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

In [ ]:
#Save as shapefile
arcpy.conversion.ExportFeatures(
    in_features="RasterT_Expand_PairwiseInter5",
    out_features=r"G:\CASA\ejen\Subcity Shapefiles\Not cleaned\surge.shp",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='FID_RasterT_Expand_6 "FID_RasterT_Expand_6" true true false 4 Long 0 0,First,#,RasterT_Expand_PairwiseInter5,FID_RasterT_Expand_6,-1,-1;Id "Id" true true false 4 Long 0 0,First,#,RasterT_Expand_PairwiseInter5,Id,-1,-1;gridcode "gridcode" true true false 4 Long 0 0,First,#,RasterT_Expand_PairwiseInter5,gridcode,-1,-1;FID_cities_dataset_final "FID_cities_dataset_final" true true false 4 Long 0 0,First,#,RasterT_Expand_PairwiseInter5,FID_cities_dataset_final,-1,-1;ID_HDC_G0 "ID_HDC_G0" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,ID_HDC_G0,-1,-1;UC_NM_MN "UC_NM_MN" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,UC_NM_MN,0,80;UC_NM_LST "UC_NM_LST" true true false 254 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,UC_NM_LST,0,254;CTR_MN_NM "CTR_MN_NM" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,CTR_MN_NM,0,80;REG_GHSL "REG_GHSL" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,REG_GHSL,0,80;REG_USAID "REG_USAID" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,REG_USAID,0,80;IN_USAID "IN_USAID" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,IN_USAID,-1,-1;P15 "P15" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P15,-1,-1;P20 "P20" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P20,-1,-1;P25 "P25" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P25,-1,-1;P20_25 "P20_25" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P20_25,-1,-1;P20_25_CAT "P20_25_CAT" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,P20_25_CAT,0,80;P20_25_BIN "P20_25_BIN" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P20_25_BIN,-1,-1;P25_CAT "P25_CAT" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,P25_CAT,0,80;P25_SMMED "P25_SMMED" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P25_SMMED,-1,-1;P_AGE_U15 "P_AGE_U15" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P_AGE_U15,-1,-1;P_AGE_65UP "P_AGE_65UP" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P_AGE_65UP,-1,-1;P_AGE_DEP "P_AGE_DEP" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P_AGE_DEP,-1,-1;GDP15 "GDP15" true true false 4 Long 0 0,First,#,RasterT_Expand_PairwiseInter5,GDP15,-1,-1;GDP15_PER "GDP15_PER" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,GDP15_PER,-1,-1;P00_20 "P00_20" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P00_20,-1,-1;P00_20_MIG "P00_20_MIG" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,P00_20_MIG,-1,-1;D_1M "D_1M" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,D_1M,-1,-1;D_1M_NM "D_1M_NM" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,D_1M_NM,0,80;D_500K "D_500K" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,D_500K,-1,-1;D_500K_NM "D_500K_NM" true true false 80 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,D_500K_NM,0,80;L_DRY "L_DRY" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,L_DRY,-1,-1;L_COAST "L_COAST" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,L_COAST,-1,-1;L_ELEV "L_ELEV" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,L_ELEV,-1,-1;RISK_TEMP_ "RISK_TEMP_" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_TEMP_,-1,-1;RISK_WBGT_ "RISK_WBGT_" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_WBGT_,-1,-1;RISK_TEMP "RISK_TEMP" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_TEMP,-1,-1;RISK_WBGT "RISK_WBGT" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_WBGT,-1,-1;RISK_SCAR "RISK_SCAR" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_SCAR,-1,-1;RISK_RIVR_ "RISK_RIVR_" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_RIVR_,-1,-1;RISK_RIVR "RISK_RIVR" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_RIVR,-1,-1;RISK_SURG_ "RISK_SURG_" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_SURG_,-1,-1;RISK_SURG "RISK_SURG" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_SURG,-1,-1;RISK_CYC1 "RISK_CYC1" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_CYC1,-1,-1;RISK_CYC2 "RISK_CYC2" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_CYC2,-1,-1;RISK_LAND "RISK_LAND" true true false 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,RISK_LAND,-1,-1;unique_id "unique_id" true true false 254 Text 0 0,First,#,RasterT_Expand_PairwiseInter5,unique_id,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,RasterT_Expand_PairwiseInter5,Shape_Area,-1,-1',
    sort_field=None
)